In [14]:
# Manager Agent
#  └─ 어떤 분석이 필요한지 판단
#  └─ Worker Agent들에게 작업 지시
#  └─ 결과를 종합해 최종 아이디어 결정

# Worker Agents
#  ├─ 시장 분석
#  ├─ 운영 난이도 분석
#  └─ 차별성 분석


# 환경변수 로딩

In [15]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Agent 정의

In [16]:
from crewai import Agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 매니저 에이전트
manager_agent = Agent(
    role="카페 창업 컨설팅 매니저",
    goal="여러 분석 결과를 종합하여 가장 적합한 카페 창업 아이디어를 결정한다",
    backstory=(
        "당신은 창업 컨설팅 회사의 팀장이다. "
        "실무자들의 분석 결과를 보고 최종 결정을 내린다."
    ),
    llm=llm,
    allow_delegation=True,   # ⭐ 핵심
)

# 실무 에이전트들
market_agent = Agent(
    role="시장 분석가",
    goal="카페 창업 아이디어를 시장 관점에서 분석한다",
    backstory="카페 트렌드와 소비자 수요를 분석하는 전문가이다.",
    llm=llm,
    allow_delegation=False,  
)

operation_agent = Agent(
    role="운영 분석가",
    goal="카페 운영 난이도와 현실성을 분석한다",
    backstory="카페 운영 경험이 풍부한 실무 전문가이다.",
    llm=llm,
    allow_delegation=False,  
)

differentiation_agent = Agent(
    role="차별성 분석가",
    goal="각 카페 아이디어의 차별성을 분석한다",
    backstory="경쟁 카페 분석을 전문으로 한다.",
    llm=llm,
    allow_delegation=False,  
)


# Task 정의

In [17]:
from crewai import Task

market_task = Task(
    description=(
        "다음 카페 창업 아이디어를 시장 관점에서 분석하시오:\n"
        "1. 무인 키오스크 카페\n"
        "2. 반려동물 동반 카페\n"
        "3. 공부/작업 전용 카페\n"
        "4. 지역 특산물 테마 카페"
    ),
    expected_output="각 아이디어의 시장 장점과 단점을 간단히 정리한 분석 결과",
    agent=market_agent,
)

operation_task = Task(
    description="각 카페 창업 아이디어의 운영 난이도와 현실성을 평가하시오.",
    expected_output="운영 관점에서의 장단점 분석",
    agent=operation_agent,
)

differentiation_task = Task(
    description="각 카페 창업 아이디어의 차별성을 비교 분석하시오.",
    expected_output="차별성 중심 비교 분석 결과",
    agent=differentiation_agent,
)

manager_task = Task(
    description=(
        "실무 에이전트들의 분석 결과를 종합하여\n"
        "가장 추천할 만한 카페 창업 아이디어 1개를 선택하고\n"
        "그 이유를 간단한 보고서 형태로 한글로 작성하시오."
    ),
    expected_output="최종 추천 아이디어와 선택 이유",
    agent=manager_agent,
)


# Crew 정의

In [18]:
from crewai import Crew, Process

cafe_startup_crew = Crew(
    agents=[
        market_agent,
        operation_agent,
        differentiation_agent,
    ],
    tasks=[
        market_task,
        operation_task,
        differentiation_task,
        manager_task,
    ],
    process=Process.hierarchical,  # ⭐ 핵심
    manager_agent=manager_agent,
    verbose=True,
    output_log_file="logs/cafe_startup_crew_run.json",
)


# 실행

In [19]:
# crew 실행
result = cafe_startup_crew.kickoff()

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  934266a2-380d-4573-ac98-4db01954a02c                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: 다음 카페 창업 아이디어를 시장 관점에서 분석하시오:                                                      │
│  1. 무인 키오스크 카페                                                                                          │
│  2. 반려동물 동반 카페                                                                                          │
│  3. 공부/작업 전용 카페                                                                                         │
│  4. 지역 특산물 테마 카페                                                                                       │
│  ID: 4a13731b-1f36-41c2-a721-ac65a77c5484                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 카페 창업 컨설팅 매니저                                                                                 │
│                                                                                                                 │
│  Task: 다음 카페 창업 아이디어를 시장 관점에서 분석하시오:                                                      │
│  1. 무인 키오스크 카페                                                                                          │
│  2. 반려동물 동반 카페                                                                                          │
│  3. 공부/작업 전용 카페                                                                                         │
│  4. 지역 특산물 테마 카페                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#9) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: Delegate work to coworker                                                                                │
│  Args: {"task": "Analyze the market advantages and disadvantages of the following cafe startup ideas: 1.        │
│  Unmanned kiosk cafe 2. Pet-friendly cafe 3. Study/work dedicated cafe 4. Local specialty theme cafe.",...      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 시장 분석가                                                                                             │
│                                                                                                                 │
│  Task: Analyze the market advantages and disadvantages of the following cafe startup ideas: 1. Unmanned kiosk   │
│  cafe 2. Pet-friendly cafe 3. Study/work dedicated cafe 4. Local specialty theme cafe.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  추천 카페 창업 아이디어: **무인 키오스크 카페**                                                                │
│                                                                                                                 │
│  선택 이유:                                                                                                     │
│  무인 키오스크 카페는 현재 시장에서 가장 높은 성공 가능성을 가진 아이디어로 판단됩니다. 다음은 그 이유입니다.   │
│                                                                                                                 │
│  1. **시장 잠재력**:                                                                                            │
│     무인 키오스크 카페는 기술을 활용하여 운영 비용을 절감하고 효율성을 극대화할 수 있는 모델입니다. 특히,       │
│  포스트 팬데믹 시대에 접어들면서 비대면 서비스에 대한 수요가 증가하고 있으며, 바쁜 도시 지역에서 빠르고 편리한  │
│  서비스를 제공할 수 있습니다.                                                                                   │
│                                                                                                                 │
│  2. **타겟 고객층**:                                                                                            │
│     이 카페는 기술에 익숙한 젊은 세대, 즉 대학생과 직장인들을 주요 고객으로 삼고 있습니다. 이들은 빠른          │
│  서비스와 편리함을 중시하며, 무인 시스템을 통해 개인화된 주문 경험을 선호합니다.                                │
│                                                                                                                 │
│  3. **운영 강점**:                                                                                              │
│     - **낮은 인건비**: 무인 시스템으로 인해 인력 비용이 크게 줄어들어 운영 효율성이 높아집니다.                 │
│     - **24시간 운영 가능**: 무인 카페는 시간 제약 없이 운영할 수 있어, 다양한 고객의 요구를 충족할 수           │
│  있습니다.                                                                                                      │
│     - **기술적 혁신**: 최신 기술을 활용하여 고객에게 새로운 경험을 제공함으로써 경쟁력을 갖출 수 있습니다.      │
│                                                                                                                 │
│  4. **운영 약점**:                                                                                              │
│     - **기술 의존성**: 기술적 문제 발생 시 운영에 차질이 생길 수 있으며, 고객 서비스의 질이 떨어질 수           │
│  있습니다.                                                                                                      │
│     - **고객 저항**: 일부 고객은 무인 서비스에 대한 불안감을 느낄 수 있습니다.                                  │
│                                                                                                                 │
│  5. **차별성**:                                                                                                 │
│     무인 키오스크 카페는 기존의 전통적인 카페와는 다른 독특한 경험을 제공합니다. 고객은 대기 없이 즉시          │
│  주문하고 결제할 수 있으며, 이는 바쁜 현대인들에게 큰 매력으로 작용합니다.                                      │
│                                                                                                                 │
│  결론적으로, 무인 키오스크 카페는 현재의 시장 환경에서 높은 성장 가능성을 지닌 아이디어로, 효율성과 편리함을    │
│  중시하는 고객의 요구를 충족할 수 있는 최적의 선택입니다. 따라서 이 아이디어를 추천합니다.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 카페 창업 컨설팅 매니저                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  추천 카페 창업 아이디어: **무인 키오스크 카페**                                                                │
│                                                                                                                 │
│  선택 이유:                                                                                                     │
│  무인 키오스크 카페는 현재 시장에서 가장 높은 성공 가능성을 가진 아이디어로 판단됩니다. 다음은 그 이유입니다.   │
│                                                                                                                 │
│  1. **시장 잠재력**:                                                                                            │
│     무인 키오스크 카페는 기술을 활용하여 운영 비용을 절감하고 효율성을 극대화할 수 있는 모델입니다. 특히,       │
│  포스트 팬데믹 시대에 접어들면서 비대면 서비스에 대한 수요가 증가하고 있으며, 바쁜 도시 지역에서 빠르고 편리한  │
│  서비스를 제공할 수 있습니다.                                                                                   │
│                                                                                                                 │
│  2. **타겟 고객층**:                                                                                            │
│     이 카페는 기술에 익숙한 젊은 세대, 즉 대학생과 직장인들을 주요 고객으로 삼고 있습니다. 이들은 빠른          │
│  서비스와 편리함을 중시하며, 무인 시스템을 통해 개인화된 주문 경험을 선호합니다.                                │
│                                                                                                                 │
│  3. **운영 강점**:                                                                                              │
│     - **낮은 인건비**: 무인 시스템으로 인해 인력 비용이 크게 줄어들어 운영 효율성이 높아집니다.                 │
│     - **24시간 운영 가능**: 무인 카페는 시간 제약 없이 운영할 수 있어, 다양한 고객의 요구를 충족할 수           │
│  있습니다.                                                                                                      │
│     - **기술적 혁신**: 최신 기술을 활용하여 고객에게 새로운 경험을 제공함으로써 경쟁력을 갖출 수 있습니다.      │
│                                                                                                                 │
│  4. **운영 약점**:                                                                                              │
│     - **기술 의존성**: 기술적 문제 발생 시 운영에 차질이 생길 수 있으며, 고객 서비스의 질이 떨어질 수           │
│  있습니다.                                                                                                      │
│     - **고객 저항**: 일부 고객은 무인 서비스에 대한 불안감을 느낄 수 있습니다.                                  │
│                                                                                                                 │
│  5. **차별성**:                                                                                                 │
│     무인 키오스크 카페는 기존의 전통적인 카페와는 다른 독특한 경험을 제공합니다. 고객은 대기 없이 즉시          │
│  주문하고 결제할 수 있으며, 이는 바쁜 현대인들에게 큰 매력으로 작용합니다.                                      │
│                                                                                                                 │
│  결론적으로, 무인 키오스크 카페는 현재의 시장 환경에서 높은 성장 가능성을 지닌 아이디어로, 효율성과 편리함을    │
│  중시하는 고객의 요구를 충족할 수 있는 최적의 선택입니다. 따라서 이 아이디어를 추천합니다.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  실무 에이전트들의 분석 결과를 종합하여                                                                         │
│  가장 추천할 만한 카페 창업 아이디어 1개를 선택하고                                                             │
│  그 이유를 간단한 보고서 형태로 한글로 작성하시오.                                                              │
│  Agent:                                                                                                         │
│  카페 창업 컨설팅 매니저                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  934266a2-380d-4573-ac98-4db01954a02c                                                                           │
│  Final Output: 추천 카페 창업 아이디어: **무인 키오스크 카페**                                                  │
│                                                                                                                 │
│  선택 이유:                                                                                                     │
│  무인 키오스크 카페는 현재 시장에서 가장 높은 성공 가능성을 가진 아이디어로 판단됩니다. 다음은 그 이유입니다.   │
│                                                                                                                 │
│  1. **시장 잠재력**:                                                                                            │
│     무인 키오스크 카페는 기술을 활용하여 운영 비용을 절감하고 효율성을 극대화할 수 있는 모델입니다. 특히,       │
│  포스트 팬데믹 시대에 접어들면서 비대면 서비스에 대한 수요가 증가하고 있으며, 바쁜 도시 지역에서 빠르고 편리한  │
│  서비스를 제공할 수 있습니다.                                                                                   │
│                                                                                                                 │
│  2. **타겟 고객층**:                                                                                            │
│     이 카페는 기술에 익숙한 젊은 세대, 즉 대학생과 직장인들을 주요 고객으로 삼고 있습니다. 이들은 빠른          │
│  서비스와 편리함을 중시하며, 무인 시스템을 통해 개인화된 주문 경험을 선호합니다.                                │
│                                                                                                                 │
│  3. **운영 강점**:                                                                                              │
│     - **낮은 인건비**: 무인 시스템으로 인해 인력 비용이 크게 줄어들어 운영 효율성이 높아집니다.                 │
│     - **24시간 운영 가능**: 무인 카페는 시간 제약 없이 운영할 수 있어, 다양한 고객의 요구를 충족할 수           │
│  있습니다.                                                                                                      │
│     - **기술적 혁신**: 최신 기술을 활용하여 고객에게 새로운 경험을 제공함으로써 경쟁력을 갖출 수 있습니다.      │
│                                                                                                                 │
│  4. **운영 약점**:                                                                                              │
│     - **기술 의존성**: 기술적 문제 발생 시 운영에 차질이 생길 수 있으며, 고객 서비스의 질이 떨어질 수           │
│  있습니다.                                                                                                      │
│     - **고객 저항**: 일부 고객은 무인 서비스에 대한 불안감을 느낄 수 있습니다.                                  │
│                                                                                                                 │
│  5. **차별성**:                                                                                                 │
│     무인 키오스크 카페는 기존의 전통적인 카페와는 다른 독특한 경험을 제공합니다. 고객은 대기 없이 즉시          │
│  주문하고 결제할 수 있으며, 이는 바쁜 현대인들에게 큰 매력으로 작용합니다.                                      │
│                                                                                                                 │
│  결론적으로, 무인 키오스크 카페는 현재의 시장 환경에서 높은 성장 가능성을 지닌 아이디어로, 효율성과 편리함을    │
│  중시하는 고객의 요구를 충족할 수 있는 최적의 선택입니다. 따라서 이 아이디어를 추천합니다.                      │
│                                                                                                                 │
│                                                                                    

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [23]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

추천 카페 창업 아이디어: **무인 키오스크 카페**

선택 이유:
무인 키오스크 카페는 현재 시장에서 가장 높은 성공 가능성을 가진 아이디어로 판단됩니다. 다음은 그 이유입니다.

1. **시장 잠재력**: 
   무인 키오스크 카페는 기술을 활용하여 운영 비용을 절감하고 효율성을 극대화할 수 있는 모델입니다. 특히, 포스트 팬데믹 시대에 접어들면서 비대면 서비스에 대한 수요가 증가하고 있으며, 바쁜 도시 지역에서 빠르고 편리한 서비스를 제공할 수 있습니다.

2. **타겟 고객층**: 
   이 카페는 기술에 익숙한 젊은 세대, 즉 대학생과 직장인들을 주요 고객으로 삼고 있습니다. 이들은 빠른 서비스와 편리함을 중시하며, 무인 시스템을 통해 개인화된 주문 경험을 선호합니다.

3. **운영 강점**: 
   - **낮은 인건비**: 무인 시스템으로 인해 인력 비용이 크게 줄어들어 운영 효율성이 높아집니다.
   - **24시간 운영 가능**: 무인 카페는 시간 제약 없이 운영할 수 있어, 다양한 고객의 요구를 충족할 수 있습니다.
   - **기술적 혁신**: 최신 기술을 활용하여 고객에게 새로운 경험을 제공함으로써 경쟁력을 갖출 수 있습니다.

4. **운영 약점**: 
   - **기술 의존성**: 기술적 문제 발생 시 운영에 차질이 생길 수 있으며, 고객 서비스의 질이 떨어질 수 있습니다.
   - **고객 저항**: 일부 고객은 무인 서비스에 대한 불안감을 느낄 수 있습니다.

5. **차별성**: 
   무인 키오스크 카페는 기존의 전통적인 카페와는 다른 독특한 경험을 제공합니다. 고객은 대기 없이 즉시 주문하고 결제할 수 있으며, 이는 바쁜 현대인들에게 큰 매력으로 작용합니다.

결론적으로, 무인 키오스크 카페는 현재의 시장 환경에서 높은 성장 가능성을 지닌 아이디어로, 효율성과 편리함을 중시하는 고객의 요구를 충족할 수 있는 최적의 선택입니다. 따라서 이 아이디어를 추천합니다.

In [21]:
import json

with open("logs/cafe_startup_crew_run.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("logs/cafe_startup_crew_run_readable.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("cafe_startup_crew_run_readable.json 생성 완료")

cafe_startup_crew_run_readable.json 생성 완료
